In [35]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
# np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../allRunsAllFractions.csv')
# In einem ersten Ansatz wurden alle Fraktion als Zahlwert mit einbezogen
conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #
]

values = [1, 2, 3]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "fraction", "time"])
len(dfPrepSample)
dfPrepSample

,total_workers,resource_mining,total_army_value,total_army,supply
1,46,28154,5825,96.0,174.0
3,72,39176,9750,126.0,200.0
4,35,22633,800,14.0,70.0
6,66,22585,2375,40.0,117.0
12,77,35585,6800,106.0,181.0
...,...,...,...,...,...
12654,80,49553,8525,110.0,200.0
12655,73,49464,8550,123.0,200.0
12659,70,26351,4625,70.0,157.0
12661,70,24263,5300,76.0,165.0


In [36]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,total_workers,resource_mining,total_army_value,total_army,supply
9875,68,48708,5800,95.0,165.0
10010,52,30857,6325,90.0,149.0
10761,61,35967,7425,124.0,200.0
4746,68,45912,8975,118.0,189.0
8220,64,28230,4150,71.0,165.0


In [37]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,total_workers,resource_mining,total_army_value,total_army,supply
10386,78,36288,7325,107.0,200.0
8607,47,9982,1700,22.0,78.0
2176,36,13743,1300,17.0,70.0
3668,71,25807,6200,79.0,157.0
11589,80,34843,8250,109.0,200.0


In [38]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,total_workers,resource_mining,total_army_value,total_army,supply
9875,68,48708,5800,95.0,165.0
10010,52,30857,6325,90.0,149.0
10761,61,35967,7425,124.0,200.0
4746,68,45912,8975,118.0,189.0
8220,64,28230,4150,71.0,165.0
...,...,...,...,...,...
2729,64,24617,6375,91.0,173.0
8147,26,6776,150,3.0,46.0
453,64,17742,4125,45.0,133.0
10659,51,24478,4950,77.0,157.0


In [39]:

dfTrainLabels = dfTrainFeatures.pop("total_army_value")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[6.8000e+01, 4.8708e+04, 9.5000e+01, 1.6500e+02],
       [5.2000e+01, 3.0857e+04, 9.0000e+01, 1.4900e+02],
       [6.1000e+01, 3.5967e+04, 1.2400e+02, 2.0000e+02],
       ...,
       [6.4000e+01, 1.7742e+04, 4.5000e+01, 1.3300e+02],
       [5.1000e+01, 2.4478e+04, 7.7000e+01, 1.5700e+02],
       [5.0000e+01, 1.8212e+04, 1.8000e+01, 8.5000e+01]])

In [40]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [41]:
sc_model = tf.keras.Sequential([layers.Dense(50, activation="relu"),layers.Dense(30, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [42]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=3000)  #350

Epoch 1/3000
90/90 [==============================] - 0s 647us/step - loss: 4256443.5000
Epoch 2/3000
90/90 [==============================] - 0s 589us/step - loss: 3805366.0000
Epoch 3/3000
90/90 [==============================] - 0s 586us/step - loss: 4214174.0000
Epoch 4/3000
90/90 [==============================] - 0s 599us/step - loss: 4031806.2500
Epoch 5/3000
90/90 [==============================] - 0s 591us/step - loss: 3822001.5000
Epoch 6/3000
90/90 [==============================] - 0s 582us/step - loss: 4034492.2500
Epoch 7/3000
90/90 [==============================] - 0s 593us/step - loss: 3716870.7500
Epoch 8/3000
90/90 [==============================] - 0s 587us/step - loss: 4045975.5000
Epoch 9/3000
90/90 [==============================] - 0s 578us/step - loss: 3796748.7500
Epoch 10/3000
90/90 [==============================] - 0s 580us/step - loss: 3535155.5000
Epoch 11/3000
90/90 [==============================] - 0s 594us/step - loss: 3365399.7500
Epoch 12/3000
90/90

In [43]:
sc_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 50)                250       
                                                                 
 dense_12 (Dense)            (None, 30)                1530      
                                                                 
 dense_13 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1,811
Trainable params: 1,811
Non-trainable params: 0
_________________________________________________________________


In [44]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("total_army_value")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

10386    7325
8607     1700
2176     1300
3668     6200
11589    8250
Name: total_army_value, dtype: int64

In [45]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    stri = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

23/23 [==============================] - 0s 497us/step
The R^2 value in the test set is 0.9628912530129077


In [46]:
sc_model.save('trained_model.h5')
